# Systems Immunogenetics Project

## QTL Mapping: Creating the Genome Cache

### McWeeney Lab, Oregon Health & Science University

#### Author: Michael Mooney (mooneymi@ohsu.edu)

## Introduction

This document will walk through the steps for mapping QTLs in the RIX lines.

Required Files:

1. This notebook** (`SIG_Genome_Cache_Workflow.ipynb`): [[Download here]](https://raw.githubusercontent.com/biodev/SIG/master/SIG_Genome_Cache_Workflow.ipynb)
2. The R script `rix_qtl_mapping_functions.r`: [[Download here]](https://raw.githubusercontent.com/biodev/SIG/master/scripts/rix_qtl_mapping_functions.r)

** Note: this notebook can also be downloaded as an R script (only the code blocks seen below will be included): [[Download R script here]](https://raw.githubusercontent.com/biodev/SIG/master/SIG_Genome_Cache_Workflow.r)

Required R Libraries:

1. `gdata`: [https://cran.r-project.org/web/packages/gdata/index.html](https://cran.r-project.org/web/packages/gdata/index.html)
2. `DOQTL`: [https://www.bioconductor.org/packages/release/bioc/html/DOQTL.html](https://www.bioconductor.org/packages/release/bioc/html/DOQTL.html)
3. `GenomicRanges`: [https://bioconductor.org/packages/release/bioc/html/GenomicRanges.html](https://bioconductor.org/packages/release/bioc/html/GenomicRanges.html)
4. `VariantAnnotation`: [https://bioconductor.org/packages/release/bioc/html/VariantAnnotation.html](https://bioconductor.org/packages/release/bioc/html/VariantAnnotation.html)
5. `foreach`: [https://cran.r-project.org/web/packages/foreach/index.html](https://cran.r-project.org/web/packages/foreach/index.html)
6. `doParallel`: [https://cran.r-project.org/web/packages/doParallel/index.html](https://cran.r-project.org/web/packages/doParallel/index.html)
7. `RColorBrewer`: [https://cran.r-project.org/web/packages/RColorBrewer/index.html](https://cran.r-project.org/web/packages/RColorBrewer/index.html)

**All code is available on GitHub: [https://github.com/biodev/SIG](https://github.com/biodev/SIG)**

## Step 1. Load Necessary R Functions and Libraries

In [1]:
## Load R functions and libraries
source('./scripts/rix_qtl_mapping_functions.r')

gdata: read.xls support for 'XLS' (Excel 97-2004) files ENABLED.

gdata: read.xls support for 'XLSX' (Excel 2007+) files ENABLED.

Attaching package: ‘gdata’

The following object is masked from ‘package:stats’:

    nobs

The following object is masked from ‘package:utils’:

    object.size

The following object is masked from ‘package:base’:

    startsWith

Loading required package: BSgenome.Mmusculus.UCSC.mm10
Loading required package: BSgenome
Loading required package: BiocGenerics
Loading required package: parallel

Attaching package: ‘BiocGenerics’

The following objects are masked from ‘package:parallel’:

    clusterApply, clusterApplyLB, clusterCall, clusterEvalQ,
    clusterExport, clusterMap, parApply, parCapply, parLapply,
    parLapplyLB, parRapply, parSapply, parSapplyLB

The following object is masked from ‘package:gdata’:

    combine

The following objects are masked from ‘package:stats’:

    IQR, mad, xtabs

The following objects are masked from ‘package:base’:

   

## Step 2. Create Genome Cache

If the RIX list below is complete, you should only have to create the cache once (one for males and one for females). You can then subset the cache based on the particular mapping population being studied (as shown in Step 6 below). If you've already created the genome cache, skip to step 3.

In [2]:
## First get all RIX matings
rix_list = read.xls('./data/BOBs_Corrected_16_06_22.xlsx', header=T, as.is=T)
rix_list = rix_list[,1:2]
colnames(rix_list) = c('Mating', 'CC_Mating')
head(rix_list)

,Mating,CC_Mating
1,477x16912,CC036xCC051
2,1566x8002,CC021xCC032
3,1566x8043,CC021xCC023
4,3015x5306,CC074xCC062
5,3015x5358,CC074xCC058
6,3015x15156,CC074xCC002


In [3]:
## Set directory containing founder 36-state probability files, and get filenames
cc_dir = '/Users/mooneymi/Documents/SIG/Mapping/CC'
cc_prob_files = list.files(cc_dir, pattern="CC...-.*b38.*\\.csv")
length(cc_prob_files)
cc_prob_files[1:5]

[1] 72

[1] "CC001-Uncb38V01.csv"    "CC002-Uncb38V01.csv"    "CC003-Uncb38V01.csv"   
[4] "CC004-TauUncb38V01.csv" "CC005-TauUncb38V01.csv"

In [4]:
## Get vectors of parental strains
rix_strains = unlist(strsplit(rix_list$Mating, 'x'))
cc_strains = unlist(strsplit(rix_list$CC_Mating, 'x'))

## Check strains (some cc_strains will be duplicated -- re-derived lines)
dup_idx = duplicated(rix_strains)
cc_strains = cc_strains[!dup_idx]
names(cc_strains) = rix_strains[!dup_idx]
length(cc_strains)
cc_strains[1:5]

[1] 69

477   16912    1566    8002    8043 
"CC036" "CC051" "CC021" "CC032" "CC023"

In [5]:
## Check that all RIX strains mapped to a CC strains
sum(is.na(cc_strains))

[1] 0

In [6]:
## Set directory containing the 8-state probability files
## These files can be created using the collapse_probs() function (see next code block)
rix_dir = '/Users/mooneymi/Documents/SIG/Mapping/RIX'

In [ ]:
## If you haven't already done so, the following code can be used to 
## create the 8-state probability files (you should only have to do this once)
## File names will be the RIX IDs (not CC IDs). 
## Note: you may have to make copies of some files for re-derived lines 
## (e.g. 8042 and 18042 are the same), if this wasn't handled above
# for (i in 1:length(cc_strains)) {
#     ## Get RIX ID
#     rix = names(cc_strains[i])
#     
#     ## Create and save the 8-state probabilities for each parental strain
#     cc_file = file.path(cc_dir, cc_prob_files[grep(paste0(cc_strains[i], '-.*\\.csv'), cc_prob_files)])
#     cc_8state = collapse_probs(cc_file)
#     
#     ## Save Y and M chromosomes separately
#     autosome_x_markers = cc_8state$marker[!cc_8state$chromosome %in% c('Y', 'M')]
#     y_m_markers = cc_8state$marker[cc_8state$chromosome %in% c('Y', 'M')]
#     file_name = file.path(rix_dir, paste0(rix, '.csv'))
#     write.table(cc_8state[autosome_x_markers, ], file=file_name, row.names=F, col.names=T, sep=',', quote=F)
#     file_name = file.path(rix_dir, paste0(rix, '_Y_M.csv'))
#     write.table(cc_8state[y_m_markers, ], file=file_name, row.names=F, col.names=T, sep=',', quote=F)
# }

In [7]:
## Create 3D probability array - Males
## Note: the number of matings included will be printed
model.probs = make_rix_model_probs(rix_list$Mating, rix_dir, 'M')
dim(model.probs)
names(dimnames(model.probs))

[1] 1
[1] 2
[1] 3
[1] 4
[1] 5
[1] 6
[1] 7
[1] 8
[1] 9
[1] 10
[1] 11
[1] 12
[1] 13
[1] 14
[1] 15
[1] 16
[1] 17
[1] 18
[1] 19
[1] 20
[1] 21
[1] 22
[1] 23
[1] 24
[1] 25
[1] 26
[1] 27
[1] 28
[1] 29
[1] 30
[1] 31
[1] 32
[1] 33
[1] 34
[1] 35
[1] 36
[1] 37
[1] 38
[1] 39
[1] 40
[1] 41
[1] 42
[1] 43
[1] 44
[1] 45
[1] 46
[1] 47
[1] 48
[1] 49
[1] 50
[1] 51
[1] 52
[1] 53
[1] 54
[1] 55
[1] 56
[1] 57
[1] 58
[1] 59
[1] 60
[1] 61
[1] 62
[1] 63
[1] 64
[1] 65
[1] 66
[1] 67
[1] 68
[1] 69
[1] 70
[1] 71
[1] 72
[1] 73
[1] 74
[1] 75
[1] 76
[1] 77
[1] 78
[1] 79
[1] 80
[1] 81
[1] 82
[1] 83
[1] 84
[1] 85
[1] 86
[1] 87
[1] 88
[1] 89
[1] 90
[1] 91
[1] 92
[1] 93
[1] 94
[1] 95
[1] 96
[1] 97
[1] 98
[1] 99
[1] 100
[1] 101
[1] 102
[1] 103
[1] 104
[1] 105
[1] 106
[1] 107
[1] 108
[1] 109
[1] 110
[1] 111
[1] 112
[1] 113
[1] 114
[1] 115
[1] 116
[1] 117
[1] 118
[1] 119
[1] 120


[1]   120     8 76606

[1] "samples"  "founders" "markers"

In [8]:
## Check model.probs object
model.probs[1,,1:5]

,UNC6,JAX00000010,JAX00240603,JAX00240610,JAX00240613
A,3.00075e-06,2.62500e-06,2.25000e-06,1.87500e-06,1.50000e-06
B,3.751e-06,3.375e-06,3.000e-06,2.625e-06,2.250e-06
C,2.292e-06,1.875e-06,1.500e-06,1.125e-06,7.500e-07
D,2.292e-06,1.875e-06,1.500e-06,1.125e-06,7.500e-07
E,0.4999923,0.4999936,0.4999947,0.4999955,0.4999962
F,0.4999948,0.4999959,0.4999970,0.4999978,0.4999985
G,7.92e-07,3.75e-07,0.00e+00,0.00e+00,0.00e+00
H,7.5075e-07,3.7500e-07,0.0000e+00,0.0000e+00,0.0000e+00


In [9]:
## Get vector of all markers
all_markers = dimnames(model.probs)[[3]]

In [10]:
## Check if any markers sum to 0
markers_zero_idx = which(apply(model.probs, 1, colSums) == 0)
markers_zero = rep(all_markers, length(samples))[markers_zero_idx]
markers_zero = markers_zero[!is.na(markers_zero)]
markers_zero = unique(markers_zero)
length(markers_zero)

[1] 6

In [11]:
## Remove markers with all zeros
model.probs = model.probs[, , setdiff(all_markers, markers_zero)]
dim(model.probs)

[1]   120     8 76600

In [12]:
## Make all probabilities non-zero
model.probs[model.probs == 0] = 1e-20
model.probs[1,,1:5]

,UNC6,JAX00000010,JAX00240603,JAX00240610,JAX00240613
A,3.00075e-06,2.62500e-06,2.25000e-06,1.87500e-06,1.50000e-06
B,3.751e-06,3.375e-06,3.000e-06,2.625e-06,2.250e-06
C,2.292e-06,1.875e-06,1.500e-06,1.125e-06,7.500e-07
D,2.292e-06,1.875e-06,1.500e-06,1.125e-06,7.500e-07
E,0.4999923,0.4999936,0.4999947,0.4999955,0.4999962
F,0.4999948,0.4999959,0.4999970,0.4999978,0.4999985
G,7.92e-07,3.75e-07,1.00e-20,1.00e-20,1.00e-20
H,7.5075e-07,3.7500e-07,1.0000e-20,1.0000e-20,1.0000e-20


In [ ]:
## Save genome cache - Males
mapping_dir = '/Users/mooneymi/Documents/SIG/Mapping'
save(model.probs, file=file.path(mapping_dir, 'rix_universal_model_prob_males_27-Jun-2016.rda'))

In [13]:
## Create 3D probability array - Females
model.probs = make_rix_model_probs(rix_list$Mating, rix_dir, 'F')
dim(model.probs)
names(dimnames(model.probs))

[1] 1
[1] 2
[1] 3
[1] 4
[1] 5
[1] 6
[1] 7
[1] 8
[1] 9
[1] 10
[1] 11
[1] 12
[1] 13
[1] 14
[1] 15
[1] 16
[1] 17
[1] 18
[1] 19
[1] 20
[1] 21
[1] 22
[1] 23
[1] 24
[1] 25
[1] 26
[1] 27
[1] 28
[1] 29
[1] 30
[1] 31
[1] 32
[1] 33
[1] 34
[1] 35
[1] 36
[1] 37
[1] 38
[1] 39
[1] 40
[1] 41
[1] 42
[1] 43
[1] 44
[1] 45
[1] 46
[1] 47
[1] 48
[1] 49
[1] 50
[1] 51
[1] 52
[1] 53
[1] 54
[1] 55
[1] 56
[1] 57
[1] 58
[1] 59
[1] 60
[1] 61
[1] 62
[1] 63
[1] 64
[1] 65
[1] 66
[1] 67
[1] 68
[1] 69
[1] 70
[1] 71
[1] 72
[1] 73
[1] 74
[1] 75
[1] 76
[1] 77
[1] 78
[1] 79
[1] 80
[1] 81
[1] 82
[1] 83
[1] 84
[1] 85
[1] 86
[1] 87
[1] 88
[1] 89
[1] 90
[1] 91
[1] 92
[1] 93
[1] 94
[1] 95
[1] 96
[1] 97
[1] 98
[1] 99
[1] 100
[1] 101
[1] 102
[1] 103
[1] 104
[1] 105
[1] 106
[1] 107
[1] 108
[1] 109
[1] 110
[1] 111
[1] 112
[1] 113
[1] 114
[1] 115
[1] 116
[1] 117
[1] 118
[1] 119
[1] 120


[1]   120     8 76606

[1] "samples"  "founders" "markers"

In [14]:
## Check model.probs object
model.probs[1,,1:5]

,UNC6,JAX00000010,JAX00240603,JAX00240610,JAX00240613
A,3.00075e-06,2.62500e-06,2.25000e-06,1.87500e-06,1.50000e-06
B,3.751e-06,3.375e-06,3.000e-06,2.625e-06,2.250e-06
C,2.292e-06,1.875e-06,1.500e-06,1.125e-06,7.500e-07
D,2.292e-06,1.875e-06,1.500e-06,1.125e-06,7.500e-07
E,0.4999923,0.4999936,0.4999947,0.4999955,0.4999962
F,0.4999948,0.4999959,0.4999970,0.4999978,0.4999985
G,7.92e-07,3.75e-07,0.00e+00,0.00e+00,0.00e+00
H,7.5075e-07,3.7500e-07,0.0000e+00,0.0000e+00,0.0000e+00


In [15]:
## Remove markers with all zeros
model.probs = model.probs[, , setdiff(all_markers, markers_zero)]
dim(model.probs)

[1]   120     8 76600

In [16]:
## Make all probabilities non-zero
model.probs[model.probs == 0] = 1e-20
model.probs[1,,1:5]

,UNC6,JAX00000010,JAX00240603,JAX00240610,JAX00240613
A,3.00075e-06,2.62500e-06,2.25000e-06,1.87500e-06,1.50000e-06
B,3.751e-06,3.375e-06,3.000e-06,2.625e-06,2.250e-06
C,2.292e-06,1.875e-06,1.500e-06,1.125e-06,7.500e-07
D,2.292e-06,1.875e-06,1.500e-06,1.125e-06,7.500e-07
E,0.4999923,0.4999936,0.4999947,0.4999955,0.4999962
F,0.4999948,0.4999959,0.4999970,0.4999978,0.4999985
G,7.92e-07,3.75e-07,1.00e-20,1.00e-20,1.00e-20
H,7.5075e-07,3.7500e-07,1.0000e-20,1.0000e-20,1.0000e-20


In [ ]:
## Save genome cache - Females
save(model.probs, file=file.path(mapping_dir, 'rix_universal_model_prob_females_27-Jun-2016.rda'))

#### Last Updated: 31-Aug-2016